#Reading data

In [1]:
import pandas as pd

df=pd.read_csv(
    'counsel_chat.csv',
    delimiter=','
)
df.head()

,Unnamed: 0,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,split
0,0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,"If everyone thinks you're worthless, then mayb...",1,2899,train
1,1,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Robin Landwehr, DBH, LPCC, NCCMental Health in...",https://counselchat.com/therapists/robin-landw...,"Hello, and thank you for your question and see...",1,3514,train
2,2,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Lee KingI use an integrative approach to treat...,https://counselchat.com/therapists/lee-king,First thing I'd suggest is getting the sleep y...,0,5,train
3,3,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Shauntai Davis-YearginPersonalized, private on...",https://counselchat.com/therapists/shauntai-da...,Therapy is essential for those that are feelin...,0,31,train
4,4,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Jordan WhiteLicensed Social Worker at Oak Root...,https://counselchat.com/therapists/jordan-white,I first want to let you know that you are not ...,0,620,train


# Preprocessing & Loading into a dataframe

In [2]:
# filtering dataframe 
# selectiong topics 
topic_list= ['depression' ,'relationships'	,'intimacy'	,'anxiety'  ,'family-conflict'  ,	'parenting' , 	'self-esteem']  
filtered_df= df[df.topic.isin(topic_list)]
len(filtered_df)

1302

In [3]:
filtered_df.loc[filtered_df['questionID']== 1]['upvotes']

23    2
24    1
25    1
26    1
27    1
28    1
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
51    0
52    0
53    0
54    0
55    0
56    0
57    0
58    0
59    0
60    0
61    0
62    0
63    0
64    0
65    0
66    0
67    0
68    0
69    0
Name: upvotes, dtype: int64

In [4]:
# sorting
filtered_df=filtered_df.sort_values(by=['upvotes'], ascending=False)
# remove duplicated questions 
filtered_df=filtered_df.drop_duplicates(subset='questionText', keep="first")
len(filtered_df)

575

In [5]:
# dividing by topic

grouped = filtered_df.groupby('topic')
depression = grouped.get_group("depression")
depression['topic'].value_counts()

# dividing by topic

relationships = grouped.get_group("relationships")


intimacy = grouped.get_group("intimacy")


anxiety = grouped.get_group("anxiety")


family_conflict= grouped.get_group("family-conflict")

parenting = grouped.get_group("parenting")


self_esteem = grouped.get_group("self-esteem")

len(depression)

132

In [6]:
# spliting the dataframe into batches 
import numpy as np 
df_split = np.array_split(depression,70)
df_split_filtered=[]
for dataframe in df_split:
    if (len(dataframe))>=2:
      df_split_filtered.append(dataframe)
for dataframe in df_split_filtered:
    print(len(dataframe))

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [7]:
texts=[]

for dataframe in df_split_filtered:

  text=""
  for index, row in dataframe.iterrows():
      text= text + " "+ row['answerText']
  texts.append(text)

len(texts)

62

In [8]:
text_len=[]
for text in texts:
  text_len.append(len(text))
print(max(text_len))

4256


In [ ]:
df_split_filtered[0]
# Question title 
# answer text 
# exact answer 
# exact answer

,Unnamed: 0,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,split
185,185,42,Why do I feel sad all the time?,I just feel sad all the time and I don't like ...,https://counselchat.com/questions/why-do-i-fee...,depression,Virginia ChowTheraspace,https://counselchat.com/therapists/virginia-chow,"Hello,While one can be sad from time to time, ...",9,261,val
325,325,134,I get depressed whenever anyone I care about i...,My last relationships have ended horribly. The...,https://counselchat.com/questions/i-get-depres...,depression,Jessica DobbsTherapist,https://counselchat.com/therapists/jessica-dobbs,"Frequently when working with clients, I utiliz...",6,202,train


In [ ]:
texts[0]

' Hello,While one can be sad from time to time, feeling sad "all the time" could be a sign of depression. If you feel sad on most days, it is worthwhile speaking to a psychologist to determine whether you suffer clinical depression. Feeling sadness is a normal response to loss, whether you lose a family, friend, job, or something you deem important in your life. However, feeling sadness all the time is a signal that you are not happy about something in your life. If it is related to your family and often relationship with family members could affect our self-esteem and self-worth, then perhaps it is worthwhile exploring what you find unhappy about your relationship with your family. Is it that you do not feel heard, supported or loved? Is it that you feel disrespected, disregarded or feel unimportant? These are just a few examples of common issues that people report that contribute to their depression. To help you to uncover your reasons for your sadness or depression and to cope with 

# Question Answering

In [ ]:
!pip install numpy scipy librosa unidecode inflect librosa transformers

     |████████████████████████████████| 235 kB 5.1 MB/s 
     |████████████████████████████████| 2.9 MB 71.0 MB/s 
     |████████████████████████████████| 3.3 MB 54.6 MB/s 
     |████████████████████████████████| 56 kB 6.5 MB/s 
     |████████████████████████████████| 895 kB 63.3 MB/s 
     |████████████████████████████████| 596 kB 75.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# length of answers
length=filtered_df['answerText'].str.len()
length=pd.DataFrame(length).rename(columns={'answerText': 'Answer Length'})
plot=pd.DataFrame(length).plot(kind='hist',title="Length Histogram",subplots=True)
figure=plot[0].get_figure()
figure.savefig('Answer_length_histogram')

# Question Answering

In [ ]:
import torch
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')

In [ ]:
import torch

In [ ]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    inputs = tokenizer(question, answer_text,truncation=True, return_tensors='pt')
    input_ids = tokenizer.encode(question, answer_text)
   

    start_positions = torch.tensor([1])
    end_positions = torch.tensor([3])
    

    # Report how long the input sequence is.
    print('Query1 has {:,} tokens.\n'.format(len(inputs)))
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Evaluate ========
    # Run our example question through the model.
    outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
                                    # token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]
   
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    #print('Predicted Answer: "' + answer + '"')
    return answer

In [ ]:
len(df_split_filtered)

62

In [ ]:
# try on one sample
j=0
count = 0 
for i in range (0, len(df_split_filtered)):
  for index,row in df_split_filtered[i].iterrows():
    print ('Predicted answer: ',answer_question(row['questionTitle'],texts[j] ))
    print('Real Answer: ',row['answerText'])
    if (answer_question(row['questionTitle'],texts[j]) in row['answerText']):
       count =count+1
  j=j+1

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Query1 has 2 tokens.

Query has 584 tokens.



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Predicted answer:  depression . if you feel sad on most days , it is worthwhile speaking to a psychologist to determine whether you suffer clinical depression . feeling sadness is a normal response to loss , whether you lose a family , friend , job , or something you deem important in your life . however , feeling sadness all the time is a signal that you are not happy about something in your life
Real Answer:  Hello,While one can be sad from time to time, feeling sad "all the time" could be a sign of depression. If you feel sad on most days, it is worthwhile speaking to a psychologist to determine whether you suffer clinical depression. Feeling sadness is a normal response to loss, whether you lose a family, friend, job, or something you deem important in your life. However, feeling sadness all the time is a signal that you are not happy about something in your life. If it is related to your family and often relationship with family members could affect our self-esteem and self-worth,

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

Predicted answer:  [CLS]
Real Answer:  Consider yourself quite normal for feeling overwhelmed and depressed about your parents divorce.   This is the most natural way to feel at this time.Depending on how old you are, and whether you live under their roof,  are dependent on their support, and are either part of the decision or not, of with whom and where you will live, start considering these points.How did you find out about the upcoming divorce?Are either of your parents reluctant to answer your questions or is it clear that neither of them want to talk about anything with you?Whatever your fears and questions about your own future, these are all real.  It is necessary for you to know about your basic future.If you are living on your own and the main problem is your inner adjustment that your family structure is completely changing, then probably a good therapist would be a great help to you now, to clarify these tensions.Sending lots of good wishes for an easy resolution to your new

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Query1 has 2 tokens.

Query has 528 tokens.



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Predicted answer:  [CLS]
Real Answer:  Hello, and thank you for your question. It certainly sounds like you don't like how things are going. Here are a few thoughts and ideas:1. If you haven't seen a primary care provider in a while, you may want to. There are health conditions that can affect your mood and energy levels, even at your age. It doesn't hurt to get checked out. If they find the cause, they may be able to treat it and improve the symptoms. In addition, most primary care providers are trained to be able to manage medication for patients who have mild, moderate, and sometimes severe depression. So, don't be surprised if your primary care doctor offers to put you on an anti-depressant if they diagnose you with depression. Medication is not the only way to treat depression, but sometimes it is necessary depending on different factors.2. You may want to consider counseling, since it is another effective way to treat depression. A counselor will help you explore the factors that

In [ ]:
count

17

### the text are almost balanced ( in terms of length)

# Fine tuned Model

In [9]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 71.4 MB/s 
     |████████████████████████████████| 895 kB 73.0 MB/s 
     |████████████████████████████████| 3.3 MB 28.1 MB/s 
     |████████████████████████████████| 56 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [10]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [11]:
from transformers import pipeline

qa_pipeline = pipeline(
    "question-answering",
    model="mrm8488/bert-small-finetuned-squadv2",
    tokenizer="mrm8488/bert-small-finetuned-squadv2"
)


Downloading:   0%|          | 0.00/463 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:

# test on one question 
qa_pipeline({
    'context': texts[0],
    'question': "I get depressed whenever anyone I care about "

})['answer']

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


'C (Consequence): You feel depressed'

In [12]:
import warnings
warnings.filterwarnings('ignore')

# try on one sample
j=0
count = 0 
for i in range (0, len(df_split_filtered)):
  for index,row in df_split_filtered[i].iterrows():
    print ('Predicted answer: ',

qa_pipeline({
    'context': texts[j],
    'question': row['questionTitle']

})['answer'])
    print('Real Answer: ',row['answerText'])
    predicted_answer=qa_pipeline({
    'context': texts[j],
    'question': row['questionTitle']

})['answer']
    if (predicted_answer in row['answerText']):
       count =count+1
       print('contained')
  j=j+1

Predicted answer:  You feel depressed
Real Answer:  Hello,While one can be sad from time to time, feeling sad "all the time" could be a sign of depression. If you feel sad on most days, it is worthwhile speaking to a psychologist to determine whether you suffer clinical depression. Feeling sadness is a normal response to loss, whether you lose a family, friend, job, or something you deem important in your life. However, feeling sadness all the time is a signal that you are not happy about something in your life. If it is related to your family and often relationship with family members could affect our self-esteem and self-worth, then perhaps it is worthwhile exploring what you find unhappy about your relationship with your family. Is it that you do not feel heard, supported or loved? Is it that you feel disrespected, disregarded or feel unimportant? These are just a few examples of common issues that people report that contribute to their depression. To help you to uncover your reason

In [13]:
count

91